In [1]:
#bibliotecas pandas de leitura de arquivos
import pandas as pd
#biblioteca do kmeans
from sklearn.cluster import KMeans
import numpy as np
#avaliar classificacao cluster
#from sklearn.metrics.cluster import adjusted_rand_score
#visualizar graficos
import matplotlib.pyplot as plt
#caracter 
import re
#countar os itens de uma lista
from collections import Counter
#separar treino, teste, validação
from sklearn.model_selection import train_test_split
# métricas de avaliação do K-Means
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix
# Método Silhouette
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
import math
from sklearn import svm
# AIX - Explanability
import lime
from lime import lime_tabular

from IPython.display import Image
from pathlib import Path

from sklearn.ensemble import RandomForestClassifier


#from sklearn_extra.cluster import KMedoids

In [12]:
# Definição de diretórios e Disco de leitura dos arquivos (Desktop ou Notebook) 
config_dirs = open("prefixo_dados.txt").readlines() 
DIR_PREFIXO = Path(config_dirs[0].strip())


DIRETORIO_PRINCIPAL =  DIR_PREFIXO / "Projeto_PA" / "Projeto_PA_validado"
DIR_BASE = DIR_PREFIXO / "Projeto_PA" / "dataset_modificado"
DIR_KMEANS = DIRETORIO_PRINCIPAL / "KMeansRandom" / "aprendizado"
DIR_RF = DIRETORIO_PRINCIPAL / "RF"


In [3]:
#leitura do dataset completo com dados cadastrais, medidas e estatísticas

dataset = pd.read_csv(DIR_BASE / "pacientes_completos_tratados.csv",sep=';', encoding="Latin1")
dataset_apos_tratamento = dataset


In [4]:
dataset

,numero_identificacao,data_nascimento,genero,idade,peso,altura,IMC,data_exame,sist 9:00,sist 9:15,...,Mediana Diastolica em 24h,AUC Sistolica em 24h,AUC Diastolica em 24h,AUC PP Daytime,AUC PP Nighttime,DP Sistolica em 24h,DP Diastolica em 24h,Picos Sistolica,Picos Diastolica,Diferença % Dipping
0,4,1923-10-05,1,94.0,85.0,160,33.2,2018-09-10 12:25:23,105.0,108.0,...,59.0,9545.5,4902.0,3748.0,841.5,16.52,11.36,11,34,0.966110
1,5,1948-10-07,1,70.0,80.0,165,29.4,2018-12-26 07:39:41,103.0,114.0,...,62.0,8586.5,5034.5,2915.0,606.0,7.51,6.65,9,23,0.969464
2,6,1999-03-23,1,19.0,100.0,197,25.8,2018-12-26 08:06:19,147.0,148.0,...,79.0,10674.0,6154.0,3721.0,751.5,12.70,13.01,19,19,1.055812
3,7,1978-01-26,0,40.0,65.0,"1,64",24.2,2018-12-26 08:19:06,140.0,139.0,...,80.0,10679.5,6420.5,3642.0,602.5,16.13,10.34,21,9,1.106757
4,8,1990-03-07,1,28.0,120.0,172,40.6,2018-12-26 08:26:59,113.0,110.0,...,78.0,9141.5,6118.0,2405.0,566.0,12.27,12.79,10,11,1.136996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19145,21786,1972-11-25,1,47.0,75.0,165,27.5,2020-09-10 16:03:08,163.0,198.0,...,116.0,12224.5,9265.0,2439.0,474.5,18.66,14.80,49,60,1.134475
19146,21787,1970-10-19,1,49.0,90.0,183,26.9,2020-09-10 16:21:02,114.0,120.0,...,85.0,10970.0,6819.0,3319.0,738.0,17.22,13.67,32,27,1.045759
19147,21788,1985-03-15,0,35.0,93.0,167,33.3,2020-09-10 16:44:53,107.0,110.0,...,62.0,9245.0,4935.5,3556.0,713.5,14.36,8.29,14,26,1.029243
19148,21792,1952-04-21,0,68.0,64.0,160,25.0,2020-09-10 17:25:50,94.0,107.0,...,68.0,8989.5,5660.0,2815.5,460.0,14.14,9.38,19,13,1.251898


In [5]:

def gera_arquivo(todos_dados_grupo, abordagem):
   # grupos = list(set(todos_dados_grupo['grupo'].values))
    
    todos_dados_grupo.to_csv(DIR_KMEANS / f"RF_approach_{abordagem}_todos_pacientes.csv", 
                        sep=";", encoding="latin1", index=False)

    
# Plota Gráfico da qtd pacientes em cada Grupos
def grafico_hist(padrao_doenca,doenca):
    classes = []
    qtds = []

    #contagem = Counter(grupos_medidas_periodo)
    contagem = Counter(padrao_doenca)

    #for i in range(0,num_otimo_clusters_medidas_periodo):
    for i in range(0,len(set(padrao_doenca))):
        classes.append(i)
        qtds.append(contagem[i])

    classes1 = [num + 1 for num in classes]
    #plt.bar(classes1,qtds,color="#a8ddb5")
    #plt.xticks(classes1)
    #plt.ylabel('Quantidade de Pacientes')
    #plt.xlabel('Classes')
    #plt.grid()
    #plt.title('Número de Pacientes x Padrão de Comportamento de PA')
    #fig = plt.gcf()
    #fig.savefig(DIR_KMEANS / 'graficos' / f'pacientes_medidas_{(doenca.replace(" ","_")).lower()}.png',format="png")
    #plt.show()
    return classes



In [17]:
def random_forest_periodo(dataset_completo,atributo='doenca'):
    if atributo == 'doenca':
        lista_atributos = list(dataset_completo.loc[:,'Hipotensao':'Whitecoat'].columns)
    else:
         lista_atributos=[atributo]
    medidas = dataset_completo.loc[:,'sist 9:00':'diast 8:45']
    
    for atributo in lista_atributos:
        arquivo = DIR_RF / f"RF_{(atributo.replace(' ','_')).lower()}.txt"
        with open(arquivo,'w') as file:
            
            padrao_atributo = dataset_completo[atributo].astype(int)



            train_ratio = 0.75
            validation_ratio = 0.15
            test_ratio = 0.10

            # separar train(75%) - 14.485, test (10%) - 7.726  and validation (15%) - 11.588
            #  O parâmetro random_state=42 garante o rastreio do numero de identificação
            x_train, x_test, y_train, y_test = train_test_split(medidas, padrao_atributo, test_size=1 - train_ratio,random_state=42)
            x_val, x_test, y_val, y_test = train_test_split(medidas, padrao_atributo, test_size=test_ratio/(test_ratio + validation_ratio),random_state=42)

            print(f"\n\n{20*'='} Atributo: {atributo} {20*'='}\n")
            print(f"\n\n{20*'='} Atributo: {atributo} {20*'='}\n",file=file)
            classes = grafico_hist(padrao_atributo,atributo)

            random_forest = RandomForestClassifier(n_estimators=50)
            %time random_forest.fit(x_train, y_train)

            y_prev_val = random_forest.predict(x_val)
            #calcular acuracia, recall e f1
            accuracy = np.round(accuracy_score(y_val, y_prev_val)*100,3)
            recall = np.round(recall_score(y_val, y_prev_val,average='macro')*100,3)
            f1 = np.round(f1_score(y_val, y_prev_val,average='macro')*100,3)
            print(f'\nVALIDAÇÃO:\nAccuracy: {accuracy} - Recall: {recall} - F1 Score: {f1}\n')
            print(f'\nVALIDAÇÃO:\nAccuracy: {accuracy} - Recall: {recall} - F1 Score: {f1}\n',file=file)
            mat=list(confusion_matrix(y_val, y_prev_val, labels=classes))
            print(pd.DataFrame(mat,index = classes, columns = classes))
            print(pd.DataFrame(mat,index = classes, columns = classes), file=file)

            y_prev_test = random_forest.predict(x_test)
            #calcular acuracia, recall e f1
            accuracy = np.round(accuracy_score(y_test, y_prev_test)*100,3)
            recall = np.round(recall_score(y_test, y_prev_test,average='macro')*100,3)
            f1 = np.round(f1_score(y_test, y_prev_test,average='macro')*100,3)
            print(f'\n\nTESTE:\nAccuracy: {accuracy} - Recall: {recall} - F1 Score: {f1}\n')
            print(f'\n\nTESTE:\nAccuracy: {accuracy} - Recall: {recall} - F1 Score: {f1}\n',file=file)
            #Matriz da Confusão
            mat=list(confusion_matrix(y_test, y_prev_test, labels=classes))
            print(pd.DataFrame(mat,index = classes, columns = classes))
            print(pd.DataFrame(mat,index = classes, columns = classes),file=file)

            classe = pd.DataFrame({'classe':random_forest.predict(medidas)})
            dados_completos = pd.concat([dataset_completo.loc[:,:'IMC'],
                                        classe,
                                        dataset_completo.loc[:,'data_exame':]],
                                       axis=1)
            gera_arquivo(dados_completos,(atributo.replace(' ','_')).lower())
    

In [18]:
random_forest_periodo(dataset_apos_tratamento,'genero')



==================== Atributo: genero ====================

Wall time: 3.67 s

VALIDAÇÃO:
Accuracy: 99.983 - Recall: 99.981 - F1 Score: 99.982

      0     1
0  6244     0
1     2  5244


TESTE:
Accuracy: 76.984 - Recall: 76.331 - F1 Score: 76.515

      0     1
0  3490   711
1  1052  2407


In [20]:
random_forest_periodo(dataset_apos_tratamento,'doenca')



==================== Atributo: Hipotensao ====================

Wall time: 2.32 s

VALIDAÇÃO:
Accuracy: 99.991 - Recall: 99.958 - F1 Score: 99.977

      0      1
0  1196      1
1     0  10293


TESTE:
Accuracy: 95.117 - Recall: 78.618 - F1 Score: 82.988

     0     1
0  409   291
1   83  6877


==================== Atributo: Sistolica Isolada ====================

Wall time: 699 ms

VALIDAÇÃO:
Accuracy: 100.0 - Recall: 100.0 - F1 Score: 100.0

       0   1
0  11476   0
1      0  14


TESTE:
Accuracy: 99.922 - Recall: 66.667 - F1 Score: 74.98

      0  1
0  7651  0
1     6  3


==================== Atributo: Diastolica Isolada ====================

Wall time: 1.01 s

VALIDAÇÃO:
Accuracy: 99.965 - Recall: 96.078 - F1 Score: 97.95

       0   1
0  11439   0
1      4  47


TESTE:
Accuracy: 99.791 - Recall: 72.414 - F1 Score: 80.9

      0   1
0  7631   0
1    16  13


==================== Atributo: Dipping ====================

Wall time: 3.22 s

VALIDAÇÃO:
Accuracy: 100.0 - Recall: 100